In [1]:
import pandas as pd
import geopy.distance as gps
import numpy as np 
SIZE = 50000
file = "~/downloads/2014_nyc_taxi_trips.csv"
RUSH_HOUR = [7, 8, 9, 16, 17, 18, 19]

In [2]:
def driver(buffer):
    #print("calculating weekday..")
    buffer.get_weekday()
    #print("calculating rush hour..")
    buffer.get_rush_hour()
    #print("calculating weekend..")
    buffer.get_weekend()
    #print("calculating day night..")
    buffer.get_day_night()
    #print("calculating distance..")
    buffer.calc_dist()
    df = buffer.save()
    #print("done..")
    return df

In [3]:
class dataLoader():
    
    def __init__(self, f):
        self.data = f.reset_index(drop=True)

    def calc_dist(self):
        pickup_loc = list((self.data.loc[i].pickup_latitude, self.data.loc[i].pickup_longitude) for i in range(0, len(self.data)))
        dropoff_loc = list((self.data.loc[i].dropoff_latitude, self.data.loc[i].dropoff_longitude) for i in range(0, len(self.data)))
        #distance = list(gps.vincenty(cor1, cor2).miles for cor1, cor2 in zip(pickup_loc, dropoff_loc))
        distance = []
        for cor1, cor2 in zip(pickup_loc, dropoff_loc):
            try:
                dis = gps.vincenty(cor1, cor2).miles
            except ValueError:
                dis = None
            distance.append(dis)
        self.data['distance'] = distance
        
    def get_weekday(self):
        self.data['weekday'] = self.data.pickup_datetime.dt.dayofweek
    
    def get_rush_hour(self):
        self.data['rush_hour'] = self.data.pickup_datetime.dt.hour.apply(lambda x: 1 if x in RUSH_HOUR else 0)
    
    def get_weekend(self):
        self.data['weekend'] = self.data.pickup_datetime.dt.dayofweek.apply(lambda x: 1 if x in [5,6] else 0)
    
    def get_day_night(self):
        self.data['day_night'] = self.data.pickup_datetime.dt.hour.apply(lambda x: 1 if x in range(6,19) else 0)
    
    def save(self):
        #self.data.to_csv("processed_data.csv")
        return self.data

In [4]:
dat = pd.DataFrame()
i = 1
for chunck in pd.read_csv(file, chunksize=SIZE, parse_dates=['pickup_datetime']):
    print("Iteration: ", i)
    buf = dataLoader(chunck)
    if len(dat) == 0:
        dat = driver(buf)
    elif len(dat) > 0:
        dat = dat.append(driver(buf))
    i += 1 

Iteration:  1


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  del sys.path[0]


Iteration:  2
Iteration:  3
Iteration:  4
Iteration:  5
Iteration:  6
Iteration:  7
Iteration:  8
Iteration:  9
Iteration:  10
Iteration:  11
Iteration:  12
Iteration:  13
Iteration:  14
Iteration:  15
Iteration:  16
Iteration:  17
Iteration:  18
Iteration:  19
Iteration:  20
Iteration:  21
Iteration:  22
Iteration:  23
Iteration:  24
Iteration:  25
Iteration:  26
Iteration:  27
Iteration:  28
Iteration:  29
Iteration:  30
Iteration:  31
Iteration:  32
Iteration:  33
Iteration:  34
Iteration:  35
Iteration:  36
Iteration:  37
Iteration:  38


/anaconda3/lib/python3.6/site-packages/geopy/point.py:405: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)


Iteration:  39
Iteration:  40
Iteration:  41
Iteration:  42
Iteration:  43
Iteration:  44
Iteration:  45
Iteration:  46
Iteration:  47
Iteration:  48
Iteration:  49
Iteration:  50
Iteration:  51
Iteration:  52
Iteration:  53
Iteration:  54
Iteration:  55
Iteration:  56
Iteration:  57
Iteration:  58
Iteration:  59
Iteration:  60
Iteration:  61
Iteration:  62
Iteration:  63
Iteration:  64
Iteration:  65
Iteration:  66
Iteration:  67
Iteration:  68
Iteration:  69
Iteration:  70
Iteration:  71
Iteration:  72
Iteration:  73
Iteration:  74
Iteration:  75
Iteration:  76
Iteration:  77
Iteration:  78
Iteration:  79
Iteration:  80
Iteration:  81
Iteration:  82
Iteration:  83
Iteration:  84
Iteration:  85
Iteration:  86
Iteration:  87
Iteration:  88
Iteration:  89
Iteration:  90
Iteration:  91
Iteration:  92
Iteration:  93
Iteration:  94
Iteration:  95
Iteration:  96
Iteration:  97
Iteration:  98
Iteration:  99
Iteration:  100
Iteration:  101
Iteration:  102
Iteration:  103
Iteration:  104
Itera

In [10]:
len(dat)

8252682

In [11]:
dat.head(3)

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,weekday,rush_hour,weekend,day_night,distance
0,0,2014-02-19 07:22:00.00000074,12.5,2014-02-19 07:22:00,-73.986430,40.760465,-73.988990,40.737075,1,2,1,0,1,1.619554
1,1,2014-12-06 20:36:22.0000008,4.0,2014-12-06 20:36:22,-73.979815,40.751902,-73.979446,40.755481,1,5,0,1,0,0.247719
2,2,2014-02-19 16:03:00.000000160,5.5,2014-02-19 16:03:00,-73.976075,40.752422,-73.981082,40.759285,1,2,1,0,1,0.541562


In [12]:
dat.to_csv("2014_nyc_taxi_trips_with_features_d.csv")

In [13]:
dat.dtypes

Unnamed: 0                    int64
key                          object
fare_amount                 float64
pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count               int64
weekday                       int64
rush_hour                     int64
weekend                       int64
day_night                     int64
distance                    float64
dtype: object

In [14]:
dat.isnull().sum()

Unnamed: 0            0
key                   0
fare_amount           0
pickup_datetime       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     4
dropoff_latitude      4
passenger_count       0
weekday               0
rush_hour             0
weekend               0
day_night             0
distance             15
dtype: int64